In [1]:
from selenium.webdriver.common.by import By
import pandas as pd
from selenium import webdriver
import time
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup as bs
import re
import nltk
import pymorphy2
from collections import Counter
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from operator import itemgetter

### Мешки слов и предложения

In [15]:
df = pd.read_excel('/Users/Downloads/Новости.xlsx')
df_full = df.fillna('пустой_отзыв')
texts = df_full['Текст поста']
morf = pymorphy2.MorphAnalyzer()

tokens_sent = []
tokens_bow = []

tags_list = ['NOUN','ADJF','ADJS','VERB','INFN','ADVB']
for text in tqdm(texts):
    tokens_0 = nltk.word_tokenize(text)
    tokens_1 = []
    for token in tokens_0:
        token_morf_inf = morf.parse(token)
        if token_morf_inf[0].tag.POS in tags_list:
            tokens_1.append(token_morf_inf[0].normal_form)

    tokens_bow.extend(tokens_1)    
    
    sentence = ' '.join(tokens_1)
    tokens_sent.append(sentence)
    
converter = TfidfVectorizer(max_features = 1500)

pd.DataFrame(tokens_sent).to_excel('ДТП_предложения.xlsx')

pd.DataFrame(dict(Counter(tokens_bow)).items()).to_excel('ДТП_мешки_слов.xlsx') 


### Кластеры

In [18]:
clusters_num = 20
df = pd.DataFrame(tokens_sent)
df_list = df[0].values.tolist()
    
tfidfconverter = TfidfVectorizer(max_features = 1500)
X = tfidfconverter.fit_transform(df_list).toarray()

lda = LatentDirichletAllocation(n_components = clusters_num,
                                learning_method = "batch",
                                max_iter = 25,
                                random_state = 0)
document_topics = lda.fit_transform(X)

sorting = np.argsort(lda.components_, axis = 1)[:, ::-1]
feature_names = np.array(list(dict(sorted(tfidfconverter.vocabulary_.items(), key = itemgetter(1))).keys()))   
clusters_tokens = []
for num in tqdm(range (0, clusters_num)):
    tokens_list = list(feature_names[list(sorting[num])])
    clusters_tokens.append(tokens_list)   
df_clust = pd.DataFrame(clusters_tokens).T

df_clust.to_excel('ДТП_кластеры.xlsx')

In [ ]:
clusters_norm_nums = [***] #сюда вписываем найденные кластеры, ранижруя по частоте встречания

clusters_solution = []

lda = LatentDirichletAllocation(n_components = clusters_num,
                                learning_method = "batch",
                                max_iter = 25,
                                random_state = 0)
document_topics = lda.fit_transform(X)

for marks_list in tqdm(document_topics):
    clusters_solution.append(marks_list[clusters_norm_nums])
y = []
for marks_list in tqdm(clusters_solution):
    m = list(marks_list)
    y.append(m.index(max(m)))
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

classifier = RandomForestClassifier(n_estimators = 1000, random_state = 0 )
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))

scores = classifier.predict_proba(tfidfconverter.transform(df_list).toarray())
df_lists = []
index = 0
for t in df_list:
    df_lists.append(list(np.append(t,scores[index])))
    index += 1

pd.DataFrame(df_lists).to_excel('ДТП_выбранные_кластеры.xlsx')